In [20]:
# Load Seurat object and create pseudobulk for MOFA analysis based on azimuth annotations

#############################################
# Prerequisites - Load Libraries

In [21]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/jove_r//lib/R/library"


In [22]:
source('MS2_Plot_Config.r')

###############################################
# Preqrequisites Configurations & Parameters

In [23]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [24]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/input_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/results/


In [25]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [26]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data/"

In [27]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [28]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results/"

In [29]:
## SC Data name

In [30]:
sc_configs = read.csv('configurations/01_Pre_Processing_SC_Data.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/01_Pre_Processing_SC_Data.csv'”


In [31]:
head(sc_configs,2)

,data_name,data_type
,<chr>,<chr>
1,Prepared_sc_Data,h5seurat


In [32]:
sc_configs = sc_configs[sc_configs$data_name != '',]

In [33]:
sc_dataset_names = unique(sc_configs$data_name)

In [34]:
sc_dataset_names

[1] "Prepared_sc_Data"

# Load Data 

## Load Seurat object

In [35]:
sc_data_list = list()

In [36]:
### Load single-cell datasets as h5seurat format; should contain the meta-columns: sample_id; cluster_id
### Raw Counts should be given in default assay

In [ ]:
###### Load the generated seurat objects
for(i in sc_dataset_names){
    source_text = paste( data_path, '/', i , '.h5seurat', sep = '')
    print(source_text)
    print(file.info(source_text)$mtime)
    rna_seurat_data = LoadH5Seurat(source_text, assays = "RNA", quietly = TRUE )
    
    sc_data_list[[i]] = rna_seurat_data
    }


[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data//Prepared_sc_Data.h5seurat"
[1] "2023-12-08 10:27:41 CET"


Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA



## Data - Checks 

In [ ]:
#head(colSums(x = rna_seurat_data, slot = "counts") )  # raw count values from original matrix

In [ ]:
#colnames(rna_seurat_data[[]]) # needs to contain cluster_id + sample_id

In [ ]:
#dim(rna_seurat_data@assays$RNA@scale.data)

In [ ]:
#DefaultAssay(rna_seurat_data)  

In [ ]:
## Add dummy group_id

In [ ]:
sc_data_list = lapply(sc_data_list, function(x){
    x$group_id = 'dummy'
    return(x)
    })

In [ ]:
head(sc_data_list[[1]]@meta.data,2)

# Data Processing

## Convert to SCE 

In [ ]:
sc_data_list = lapply(sc_data_list, function(x){
    as.SingleCellExperiment(x)
    })
    

In [ ]:
sc_data_list[[1]]

## Check amount of cells per sample and cluster 

In [ ]:
### Amount of cells per cluster
lapply(sc_data_list, function(x){
    colSums(table(x$cluster_id,x$cluster_id ))
    })

In [ ]:
### Save as dataframe

In [ ]:
cells_per_sample_cluster = lapply(sc_data_list, function(x){
    as.data.frame( t(table(x$cluster_id, x$sample_id)))
    })

In [ ]:
### Rename columns

In [ ]:
cells_per_sample_cluster = lapply(cells_per_sample_cluster, function(x){
    colnames(x) = c('sample_id', 'cluster_id', 'Freq')
    return(x)
    })

In [ ]:
head(cells_per_sample_cluster[[1]])

In [ ]:
names(cells_per_sample_cluster)

In [ ]:
result_path

In [ ]:
### Save
for(i in names(cells_per_sample_cluster)){
    write.csv(cells_per_sample_cluster[[i]], paste0(result_path, '/01_results/01_', i, '_Cell_Sample_Cluster_Distribution.csv'))
    }
    

In [ ]:
### Visualize

In [ ]:
## Categorize the amount of cells

In [ ]:
cells_per_sample_cluster = lapply(cells_per_sample_cluster, function(x){
    colnames(x) =  c('Sample', 'Cluster_Cell_Type', 'amount_cells')
    x$amount_cells_cat[(x$amount_cells <= 3)  ] = '0-3'
    x$amount_cells_cat[(x$amount_cells <= 10)  & (x$amount_cells >3 )] = '3-10'
    x$amount_cells_cat[(x$amount_cells <= 20)  & (x$amount_cells >10 )] = '10-20'
    x$amount_cells_cat[(x$amount_cells <= 50)  & (x$amount_cells >20 )] = '20-50'
    x$amount_cells_cat[(x$amount_cells > 50)  ] = '> 50'
    return(x)
    })

In [ ]:
head(cells_per_sample_cluster[[1]],2)

In [ ]:
options(repr.plot.width=30, repr.plot.height=15)
g = lapply(cells_per_sample_cluster, function(x){
    ggplot(x, aes(Sample, Cluster_Cell_Type, fill= amount_cells_cat))  + geom_tile()   + ggtitle(label = "Amount cells per cluster and sample")+ #theme(axis.text.x=element_text(size=14, angle = 90), axis.text.y=element_text(size=14, angle = 0),axis.title=element_text(size=14,face="bold")) + 
    scale_fill_manual(name = "Amount cells", breaks=c( '0-3','3-10', 
                                 '10-20', '20-50', '> 50'), values =c("red4","rosybrown1",  "seashell", "palegreen", "palegreen4")) + plot_config + theme(axis.text.x = element_blank())
    })
# Visualize for first dataset
g[[1]]

In [ ]:
names(g)

In [ ]:
## Save figures for later inspection 


In [ ]:
figure_name = "FIG01_Amount_of_Cells_per_Sample_and_Cell_Type"

In [ ]:
# Sizes of the plot
width_par = 8
height_par = 5

In [ ]:
pdf(paste0('figures/01_figures/', figure_name, i,  '.pdf'), width =width_par, height =height_par)
for(i in names(g)){
    print(g[[i]] )
    dev.off()
    }

## Calculate gene expression percentages per cluster for thresholding

In [ ]:
gene_list = list()

In [ ]:
gene_cell_expr = list()

In [ ]:
gene_cell_expr_data = data.frame()

In [ ]:
clusters = lapply(sc_data_list, function(x){
    unique(x$cluster_id)
    })

In [ ]:
clusters[[1]]

In [ ]:
gene_expr_data = lapply(sc_data_list, function(x){
    clusters = unique(x$cluster_id)
    for(i in clusters){
        # subset data on cluster
        rna_sce_subset= x[,x$cluster_id== i] # cluster
        amount_cells = dim(rna_sce_subset)[2]

        # Calculate percentage of cells expressing gene
        amount_cells_expressing_gene = rowSums(assay(rna_sce_subset) > 0 )
        perc_cells_expressing_gene = (amount_cells_expressing_gene/ amount_cells) * 100

        gene_cell_expr[[i]] = data.frame(perc_cells_expressing_gene = perc_cells_expressing_gene, total_amount_cells_expressing_gene = amount_cells_expressing_gene)
        
        # Format as data frame
        data = gene_cell_expr[[i]]
        data$gene = rownames(gene_cell_expr[[i]])
        data$cluster = i
        gene_cell_expr_data = rbind( gene_cell_expr_data, data)
        
        

        }
    return(gene_cell_expr_data)

})

    
    

In [ ]:
head(gene_expr_data[[1]],2)

In [ ]:
unique(gene_expr_data[[1]]$cluster)

In [ ]:
### Save results for later filtering
for(i in names(gene_expr_data)){
    write.csv(gene_expr_data[[i]], paste0(result_path, '/01_results/01_', i, '_Gene_Expr_per_Cell_Type', '.csv'))
    }
    

## Aggregate to pseudobulk

In [ ]:
### Add cluster and sample-id to use for aggregation

In [ ]:
sc_data_list = lapply(sc_data_list, function(x){   

    #### Add cluster_id, sample_id and group_id columns
    x = prepSCE(x, 
        kid = 'cluster_id', # subpopulation assignments
        gid = 'group_id',  # group IDs (ctrl/stim)   # sample_id; using dummy sample id which corresponds to cluster columns
        sid = 'sample_id',   # sample IDs (ctrl/stim.1234)
        drop = FALSE)  # drop all other colData columns
    })


In [ ]:
sc_data_list[[1]]

In [ ]:
pb = lapply(sc_data_list, function(x){
        aggregateData(x,
        assay = "counts", fun = "mean",
        by = c("cluster_id", "sample_id"))
    })
# one sheet per subpopulation will be generated

In [ ]:
pb[[1]]

In [ ]:
### Save aggregated data

In [ ]:
for(i in names(pb)){
    save(  pb , file = paste0(result_path, '/01_results/01_', i, 'Pseudobulk_.RDS'))
    }

## Convert to Table Format (CSV)

In [ ]:
sc_data = data.frame()

In [ ]:
for(j in 1:length(pb)){
    dataset = names(pb)[j]
    
    for(i in names(assays(pb[[j]]))){
        data = assay(pb[[j]], i)
        
        data = as.data.frame(t(data))
        data$type = i
        
        data$dataset = dataset
        data$sample_id = rownames(data)
        data = melt(data, id.vars = c('sample_id', 'type', 'dataset'))
        
        sc_data = rbind(sc_data,data)
       
        
        }
    }
    

In [ ]:
head(sc_data,2)

In [ ]:
write.csv( sc_data, paste0(result_path, '/01_results/01_Prepared_sc_DataPseudobulk_Table', '.csv'))